In [10]:
import pandas as pd
import altair as alt

In [11]:
df1 = pd.read_csv('class_predictions_2019_batch_1.csv')
df2 = pd.read_csv('class_predictions_2019_batch_2.csv')
df3 = pd.read_csv('class_predictions_2019_batch_3.csv')

predictions = df1.append(df2).append(df3)

/tmp/ipykernel_277/2363521379.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = df1.append(df2).append(df3)


In [12]:
predictions['obs_id'] = [int(x[:-4]) for x in predictions.filename]

In [13]:
observations = pd.read_csv('observations-285599.csv')

/tmp/ipykernel_277/1236470297.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  observations = pd.read_csv('observations-285599.csv')


In [14]:
obs = observations[['id','observed_on',
                    'latitude','longitude',
                    'species_guess','image_url','url']]
pred = predictions[['obs_id','prediction','score']]

In [15]:
pred = pd.merge(pred, obs, left_on = 'obs_id', right_on = 'id', how = 'left')

In [21]:
pred['date'] = pd.to_datetime(pred['observed_on'], infer_datetime_format=True)

In [32]:
ne = pred[(pred.latitude > 42.6) & 
     (pred.latitude < 45.1) & 
     (pred.longitude > -73.5) & 
     (pred.longitude < -70.8)]

In [33]:
group = ne.groupby([ne.date.dt.isocalendar().week,ne['prediction']])['prediction'].count().reset_index(name='count')

In [36]:
bar = alt.Chart(group).mark_bar().encode(
    x=alt.X('week', scale=alt.Scale(domain=[5, 47])),
    y='count:Q',
    color='prediction:N'
)
bar

alt.Chart(...)

In [16]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [20]:
from vega_datasets import data

airports = data.airports()
states = alt.topo_feature(data.us_10m.url, feature='states')

# US states background
source = alt.topo_feature(data.world_110m.url, 'countries')

base = alt.Chart(source).mark_geoshape(
    fill='#666666',
    stroke='white'
).properties(
    width=500,
    height=300
).project('albersUsa')

points = alt.Chart(pred[pred.prediction!='adult']).mark_circle(
    size=10,
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='prediction:N',#alt.Color('date:T',
                    # scale=alt.Scale(range=['blue','red'])
                   # ),
    tooltip=['prediction', 'score', 'latitude','longitude','observed_on']
).project('albersUsa')
base + points

alt.LayerChart(...)

In [38]:
pred[pred.prediction=='egg'][:10]

,obs_id,prediction,score,id,observed_on,latitude,longitude,species_guess,image_url,url,date
118,29918393,egg,64.854074,29918393,2019-07-31,43.039382,-83.609594,Monarch,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/29918393,2019-07-31
225,27944578,egg,52.697712,27944578,2019-06-14,39.897374,-75.347586,Monarch,https://static.inaturalist.org/photos/43526322...,https://www.inaturalist.org/observations/27944578,2019-06-14
253,29825052,egg,98.251098,29825052,2019-07-29,41.566283,-72.056263,Monarch,https://static.inaturalist.org/photos/46567257...,https://www.inaturalist.org/observations/29825052,2019-07-29
511,30072779,egg,88.459283,30072779,2019-08-02,40.774487,-77.797280,Monarch,https://static.inaturalist.org/photos/46962627...,https://www.inaturalist.org/observations/30072779,2019-08-02
596,31376107,egg,99.987984,31376107,2019-08-23,42.991393,-76.370247,Danaus plexippus,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/31376107,2019-08-23
680,30147650,egg,99.979764,30147650,2019-08-03,40.794609,-73.955732,Monarch,https://static.inaturalist.org/photos/47083615...,https://www.inaturalist.org/observations/30147650,2019-08-03
688,29005228,egg,72.323215,29005228,2019-07-17,29.462921,-98.473467,Monarch,https://static.inaturalist.org/photos/45246247...,https://www.inaturalist.org/observations/29005228,2019-07-17
1028,88459343,egg,99.999368,88459343,2019-07-21,44.211080,-69.063844,Monarch,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/88459343,2019-07-21
1258,36035329,egg,77.973771,36035329,2019-11-24,32.722069,-116.992462,Monarch,https://static.inaturalist.org/photos/56909782...,https://www.inaturalist.org/observations/36035329,2019-11-24
1349,30206005,egg,75.516671,30206005,2019-08-04,43.900693,-78.604829,Monarch,https://static.inaturalist.org/photos/47178317...,https://www.inaturalist.org/observations/30206005,2019-08-04
